In [37]:
import pandas as pd
# Load the data
DATA_DIR = '../data/server_data/'
OUT_DIR = DATA_DIR+'sanitized_data/'

### Part 1

In [28]:
summative_data = pd.read_csv(DATA_DIR + 'summative_writing.tsv', sep='\t')
participant_data = pd.read_csv(DATA_DIR + 'prolific_demographic_phase1.csv')

In [32]:
### Only valid IDs
prolific_ids = participant_data[participant_data.Status=="AWAITING REVIEW"]["Participant id"]

In [33]:
print(f"Number of prolific ids: {len(prolific_ids)}")

Number of prolific ids: 116


In [34]:
scene_count = summative_data[summative_data.prolific_id.isin(prolific_ids)].groupby(['prolific_id']).scenario_num.nunique()
scene_count[scene_count>1]

prolific_id
572d05c688902e00091e4a2c    6
589f4b4b4d580c0001e0a155    6
58b20110c2cfc500016b11c7    6
59c1a06c95bc620001575b53    6
5bcc26ba4f22df00010f166d    6
                           ..
66a79572e8487718ad4f4f8d    6
66a948d79703d361dab7075b    6
66acd248c28deb9104288edb    6
66ace12c42d9a67100360274    6
66af7e7a640964091ad3ec4f    6
Name: scenario_num, Length: 116, dtype: int64

In [35]:
summative_data[summative_data.prolific_id=="5dee44454d7bea4e3f4b44a6"]

,_id,dat1,dat2,dat3,dat4,scenario_num,incident_id,prolific_id,timestamp,context_behav,context_pers
104,66b3edff880b311d8ad2da48,Sympathy,Anger,They would have put themselves in the customer...,Hang in their. We deal with these kind of cust...,0,33.0,5dee44454d7bea4e3f4b44a6,2024-08-07 21:58:23.752,NaN,NaN


In [38]:
### Only store data for valid IDs
clean_data = summative_data[summative_data.prolific_id.isin(prolific_ids)]
clean_data.to_csv(OUT_DIR+'summative_writing.tsv', sep='\t', index=False)

### Part 2

In [87]:
summative_data = pd.read_csv(DATA_DIR + 'summative_scoring.tsv', sep='\t')
participant_data = pd.read_csv(DATA_DIR + 'prolific_demographic_phase2.csv')
### Only valid IDs
prolific_ids = participant_data[(participant_data.Status == "AWAITING REVIEW") & (~participant_data['Completion code'].isna())]["Participant id"]
print(f"Number of prolific ids: {len(prolific_ids)}")

Number of prolific ids: 143


In [88]:
columns_reorder = [
    '_id', 'prolific_id', 'timestamp','dat1',
    'scenario_num', 'incident_id',
    'msg_0', 'msg_0_src', 'msg_1', 'msg_1_src', 'msg_2', 'msg_2_src', 'sincerity_0', 'sincerity_1', 'sincerity_2',
    'compassion_0', 'compassion_1', 'compassion_2', 
    'warmth_0', 'warmth_1','warmth_2',
    'actionable_2', 'actionable_1', 'actionable_0',
    'relatability_0', 'relatability_1', 'relatability_2', 
]

In [89]:
### Only store data for valid IDs
clean_data = summative_data[summative_data.prolific_id.isin(prolific_ids)]
clean_data = clean_data[columns_reorder]

In [90]:
clean_data_vanilla = clean_data[clean_data.msg_2.isna()]
clean_data_vanilla = clean_data_vanilla.dropna(axis=1, how='all')
clean_data_vanilla.to_csv(OUT_DIR+'summative_scoring_vanilla.tsv', sep='\t', index=False)

clean_data_context = clean_data[~clean_data.msg_2.isna()]
clean_data_context = clean_data_context.dropna(axis=1, how='all')
clean_data_context.to_csv(OUT_DIR+'summative_scoring_contextual.tsv', sep='\t', index=False)

In [91]:
columns_scoring_ctx = [
    'sincerity_0', 'sincerity_1', 'sincerity_2',
    'compassion_0', 'compassion_1', 'compassion_2',
    'warmth_0', 'warmth_1','warmth_2',
    'actionable_2', 'actionable_1', 'actionable_0',
    'relatability_0', 'relatability_1', 'relatability_2',
]

In [92]:
clean_data_context['total_0'] = clean_data_context['sincerity_0']+clean_data_context['compassion_0']+clean_data_context['warmth_0']+clean_data_context['actionable_0']+clean_data_context['relatability_0']
clean_data_context['total_1'] = clean_data_context['sincerity_1']+clean_data_context['compassion_1']+clean_data_context['warmth_1']+clean_data_context['actionable_1']+clean_data_context['relatability_1']
clean_data_context['total_2'] = clean_data_context['sincerity_2']+clean_data_context['compassion_2']+clean_data_context['warmth_2']+clean_data_context['actionable_2']+clean_data_context['relatability_2']

In [93]:
print(f"Sincerity | Human_ctx: {clean_data_context.sincerity_0.mean()}; AI_ctx: {clean_data_context.sincerity_1.mean()}; AI: {clean_data_context.sincerity_2.mean()}")
print(f"Compassion | Human_ctx: {clean_data_context.compassion_0.mean()}; AI_ctx: {clean_data_context.compassion_1.mean()}; AI: {clean_data_context.compassion_2.mean()}")
print(f"Warmth | Human_ctx: {clean_data_context.warmth_0.mean()}; AI_ctx: {clean_data_context.warmth_1.mean()}; AI: {clean_data_context.warmth_2.mean()}")
print(f"Actionable | Human_ctx: {clean_data_context.actionable_0.mean()}; AI_ctx: {clean_data_context.actionable_1.mean()}; AI: {clean_data_context.actionable_2.mean()}")
print(f"Relatability | Human_ctx: {clean_data_context.relatability_0.mean()}; AI_ctx: {clean_data_context.relatability_1.mean()}; AI: {clean_data_context.relatability_2.mean()}")
print(f"Total | Human_ctx: {clean_data_context.total_0.mean()}; AI_ctx: {clean_data_context.total_1.mean()}; AI: {clean_data_context.total_2.mean()}")

Sincerity | Human_ctx: 0.894927536231884; AI_ctx: 1.4275362318840579; AI: 1.4329710144927537
Compassion | Human_ctx: 0.7681159420289855; AI_ctx: 1.3224637681159421; AI: 1.309782608695652
Warmth | Human_ctx: 0.7663043478260869; AI_ctx: 1.2626811594202898; AI: 1.2735507246376812
Actionable | Human_ctx: 0.5670289855072463; AI_ctx: 1.4202898550724639; AI: 1.4818840579710144
Relatability | Human_ctx: 0.9637681159420289; AI_ctx: 1.2934782608695652; AI: 1.3278985507246377
Total | Human_ctx: 3.960144927536232; AI_ctx: 6.726449275362318; AI: 6.826086956521739


In [95]:
from scipy.stats import mannwhitneyu, kruskal, wilcoxon, ttest_rel

# Identify the columns to check for missing values
column_pairs = [
    ('sincerity_0','sincerity_1'),
    ('compassion_0','compassion_1'),
    ('warmth_0','warmth_1'),
    ('actionable_0','actionable_1'),
    ('relatability_0','relatability_1'),
    ('total_0','total_1')
]

# Initialize a list to store the results
results = []

# Perform the Mann-Whitney U test for each pair of columns
for col1, col2 in column_pairs:
    stat, p_value = ttest_rel(clean_data_context[col1], clean_data_context[col2])
    results.append({'pair': f'{col1}_vs_{col2}', 'stat': stat, 'p_value': p_value})

# Convert the results to a DataFrame
df_results = pd.DataFrame(results)

# Display the results
print(df_results)

                               pair       stat       p_value
0        sincerity_0_vs_sincerity_1  -7.363124  6.583300e-13
1      compassion_0_vs_compassion_1  -7.111043  3.591197e-12
2              warmth_0_vs_warmth_1  -6.463183  2.259015e-10
3      actionable_0_vs_actionable_1 -10.266026  9.621561e-23
4  relatability_0_vs_relatability_1  -4.428430  1.145120e-05
5                total_0_vs_total_1  -8.692603  4.062043e-17
